In [ ]:
import pandas as pd
import json
from textblob import TextBlob

In [ ]:
tweets = []
for line in open('../input/leena.json', 'r'):
    tweets.append(json.loads(line))

In [ ]:
df=pd.DataFrame(tweets)
df.head()

In [ ]:
df1=df[['reviewText','summary']]

In [ ]:
df1.head()

In [ ]:
def detect_polarity(text):
    return TextBlob(text).sentiment.polarity
df1['rating'] = df1.summary.apply(detect_polarity)
df1.head()

In [ ]:
thres=0
df1.loc[df1['rating'] > thres, 'rating'] = 1
df1.loc[df1['rating'] < thres, 'rating'] = 0

In [ ]:
import string
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
import spacy
punctuations = string.punctuation

nlp = spacy.load('en_core_web_sm')
stop_words = spacy.lang.en.stop_words.STOP_WORDS

parser = English()


In [ ]:
def spacy_tokenizer(sentence):
   
    mytokens = parser(sentence)

    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]

    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]

    return mytokens

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline

class predictors(TransformerMixin):
    def transform(self, X, **transform_params):
        
        return [clean_text(text) for text in X]

    def fit(self, X, y=None, **fit_params):
        return self

    def get_params(self, deep=True):
        return {}


def clean_text(text):
    
    return text.strip().lower()

In [ ]:
import string
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English

punctuations = string.punctuation

nlp = spacy.load('en')
stop_words = spacy.lang.en.stop_words.STOP_WORDS

parser = English()

def spacy_tokenizer(sentence):
    mytokens = parser(sentence)

    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]

    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]


    return mytokens

In [ ]:
tfidf_vector = TfidfVectorizer(tokenizer = spacy_tokenizer,ngram_range=(2,2))

In [ ]:
from sklearn.model_selection import train_test_split

X = df1['reviewText'] 
Y = df1['rating'] 

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3)

In [ ]:
from sklearn.svm import SVC
classifier = SVC()


pipe = Pipeline([("cleaner", predictors()),
                 ('vectorizer', tfidf_vector),
                 ('classifier', classifier)])


pipe.fit(X_train,Y_train)

In [ ]:
from sklearn import metrics

Y_Pred = pipe.predict(X_test)

